In [ ]:
!pip install thundersvm

     |████████████████████████████████| 507 kB 5.3 MB/s 


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0

--2021-12-07 05:16:19--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2021-12-07 05:16:19--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?uDGj9aaJi1bwCG0aXTHlZycC7ghtoAJU9bRvmxmhFTL1wx3SfYRfp074DBHXjTK9j8mds4lxeRS1FveFtWrKPo25kramU

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import thundersvm
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
filepath = "/content/gdrive/MyDrive/iot_23_data/iot23_combined (2).csv"

In [ ]:
df = pd.read_csv(filepath, nrows=800000)

In [ ]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,93513,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
799996,93514,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
799997,93515,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
799998,93516,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
del df['Unnamed: 0']

In [ ]:
df.shape

(800000, 25)

In [ ]:
df['label'].value_counts()

8    419007
0    202059
7    149900
5     24959
9      3814
2       134
1        97
4        30
Name: label, dtype: int64

In [ ]:
df.columns = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']

In [ ]:
for col in df.columns: 
    print(col) 

duration
orig_bytes
resp_bytes
missed_bytes
orig_pkts
orig_ip_bytes
resp_pkts
resp_ip_bytes
label
proto_icmp
proto_tcp
proto_udp
conn_state_OTH
conn_state_REJ
conn_state_RSTO
conn_state_RSTOS0
conn_state_RSTR
conn_state_RSTRH
conn_state_S0
conn_state_S1
conn_state_S2
conn_state_S3
conn_state_SF
conn_state_SH
conn_state_SHR


In [ ]:
print(df.columns.tolist())

['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']


In [ ]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']

In [ ]:
# SVM_classifier = SVC()
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
start = time.time()
print('program start...')
print()

svc_gpu = thundersvm.SVC(C=10, kernel='rbf')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)
svc_gpu.fit(X_train, Y_train)
print()

print('Prediction:')
y_pred = svc_gpu.predict(X_test)
print(y_pred)
print()

print('Score:')
score = svc_gpu.score(X_test,Y_test)
print(score)
print()
# accuracy_svc = svc_gpu.score(X_test, Y_test)

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...


Prediction:
[8. 5. 0. ... 5. 5. 8.]

Score:
0.5908375

program end...

time cost: 
583.6238484382629 seconds


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:

print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :
              precision    recall  f1-score   support

           0       0.62      0.96      0.75     40523
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        30
           4       0.00      0.00      0.00         8
           5       0.25      1.00      0.40      5000
           7       0.75      0.00      0.00     30088
           8       0.65      0.60      0.62     83545
           9       0.94      0.99      0.97       788

    accuracy                           0.59    160000
   macro avg       0.40      0.44      0.34    160000
weighted avg       0.65      0.59      0.53    160000



In [ ]:
# accuracy_svc

In [ ]:
# start = time.time()
# print('program start...')
# print()

# SVM_classifier = SVC(C=1.0, cache_size=1500, verbose=True).fit(X_train, Y_train)
# print()
# print(SVM_classifier.score(X_test, Y_test))
# print()

# y_pred = SVM_classifier.predict(X_test)
# print(y_pred)
# print()

# end = time.time()
# print('program end...')
# print()
# print('time cost: ')
# print(end - start, 'seconds')